In [1]:
import keras as K
import tensorflow as tf
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense, InputLayer, Dropout, Conv2D, MaxPooling2D, Flatten, Input, LSTM, Subtract

from keras.models import Model

from keras.applications.vgg16 import VGG16


In [5]:
def get_siamese_network(input_shape):
    
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    VGG = VGG16(weights='imagenet', include_top=False)
    
    for layer in VGG.layers[:10]:
        layer.trainable=False
    
    for layer in VGG.layers:
        sp='     '
        print(layer.name, sp, layer.trainable)

    left_encode = VGG(left_input)
    right_encode = VGG(right_input)
    
    net = Subtract()([left_encode, right_encode])
    net = Flatten()(net)
    output = Dense(128, activation='relu')(net)
      
    siamese_net = Model(inputs=[left_input, right_input], outputs=output)
    
    return(siamese_net)
    

In [6]:
model = get_siamese_network([480, 640, 3])

model.summary()

input_6       False
block1_conv1       False
block1_conv2       False
block1_pool       False
block2_conv1       False
block2_conv2       False
block2_pool       False
block3_conv1       False
block3_conv2       False
block3_conv3       False
block3_pool       True
block4_conv1       True
block4_conv2       True
block4_conv3       True
block4_pool       True
block5_conv1       True
block5_conv2       True
block5_conv3       True
block5_pool       True
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 480, 640, 3) 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 480, 640, 3) 0                                            
________________________________